<a href="https://colab.research.google.com/github/santoshdhalwalkar/ocrproject/blob/main/imgread.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive


In [ ]:
#import drive from google
from google.colab import drive

In [ ]:
# mount the drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install opencv-python
!pip install pytesseract
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.5/139.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 100.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 10.1 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import numpy as np
import pytesseract
from PIL import Image
import csv

In [ ]:
# Install necessary libraries
!pip install opencv-python
!pip install pytesseract

In [ ]:
!mkdir -p /content/drive/MyDrive/model
!wget -P /content/drive/MyDrive/model https://pjreddie.com/media/files/yolov3.weights
!wget -P /content/drive/MyDrive/model https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg?raw=true -O /content/drive/MyDrive/model/yolov3.cfg
!wget -P /content/drive/MyDrive/model https://github.com/pjreddie/darknet/blob/master/data/coco.names?raw=true -O /content/drive/MyDrive/model/coco.names

--2025-03-10 04:07:43--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘/content/drive/MyDrive/model/yolov3.weights.1’

yolov3.weights.1    100%[===================>] 236.52M  45.0MB/s    in 5.5s    

2025-03-10 04:07:49 (43.0 MB/s) - ‘/content/drive/MyDrive/model/yolov3.weights.1’ saved [248007048/248007048]

--2025-03-10 04:07:49--  https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg?raw=true
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/pjreddie/darknet/raw/refs/heads/master/cfg/yolov3.cfg [following]
--2025-03-10 04:07:50--  https://github.com/pjreddie/darknet/raw/refs/heads/master/cfg

In [ ]:
!sudo apt install tesseract-ocr
!sudo apt install libtesseract-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (2,756 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [ ]:
# Path to the folder containing images
IMAGE_FOLDER = '/content/drive/MyDrive/ocrimages'

In [ ]:
# YOLOv3 Configuration
YOLO_WEIGHTS = '/content/drive/MyDrive/model/yolov3.weights'
YOLO_CONFIG = '/content/drive/MyDrive/model/yolov3.cfg'
YOLO_NAMES = '/content/drive/MyDrive/model/coco.names'

In [ ]:
# Load YOLOv3 model
net = cv2.dnn.readNet(YOLO_WEIGHTS, YOLO_CONFIG)

In [ ]:
# Function to preprocess image
def preprocess_image(image):
    image = cv2.resize(image, None, fx=3, fy=3)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    processed = cv2.bitwise_not(thresh)
    return processed

# Function to extract text directly from the entire image
def extract_text(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not load image from {image_path}. Please check the path and file.")
        return ""
    processed = preprocess_image(image)
    text = pytesseract.image_to_string(processed)
    return text

# Function to save text to a CSV file with image names
def save_to_csv(text_data, image_names, output_file):
    with open(output_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Image_Name', 'Extracted_Text'])
        for image_name, text in zip(image_names, text_data):
            writer.writerow([image_name, text])

# Main workflow
def main(image_names):
    output_file = os.path.expanduser('/content/drive/MyDrive/output1.csv')  # Save on desktop
    all_text_data = []
    for image_name in image_names:
        image_path = os.path.join(IMAGE_FOLDER, image_name)
        text = extract_text(image_path)
        all_text_data.append(text)
    save_to_csv(all_text_data, image_names, output_file)
    print(f"Output saved to: {output_file}")


if __name__ == '__main__':
    image_names = [
        '/content/drive/MyDrive/ocrimages/thyrocare_0_1861.jpg',
        '/content/drive/MyDrive/ocrimages/thyrocare_0_2841.jpg',
        '/content/drive/MyDrive/ocrimages/thyrocare_0_657.jpg'
    ]
    main(image_names)




Output saved to: /content/drive/MyDrive/output1.csv


In [ ]:
# Function to preprocess image
def preprocess_image(image):
    image = cv2.resize(image, None, fx=3, fy=3)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    processed = cv2.bitwise_not(thresh)
    return processed

# Function to extract text directly from the entire image
def extract_text(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not load image from {image_path}. Please check the path and file.")
        return ""
    processed = preprocess_image(image)
    text = pytesseract.image_to_string(processed)
    return text

# Function to save text to a CSV file
def save_to_csv(text_data, output_file):
    with open(output_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Extracted_Text'])
        for line in text_data:
            writer.writerow([line])

# Main workflow
def main(image_names):
    # Changed to a path accessible in Colab
    output_file = '/content/drive/MyDrive/output.csv'
    all_text_data = []
    for image_name in image_names:
        image_path = image_name # image_names already contain full paths
        text = extract_text(image_path)
        all_text_data.append(text)
    save_to_csv(all_text_data, output_file)
    print(f"Output saved to: {output_file}")

if __name__ == '__main__':
    image_names = [
        '/content/drive/MyDrive/ocrimages/thyrocare_0_1861.jpg',
        '/content/drive/MyDrive/ocrimages/thyrocare_0_2841.jpg',
        '/content/drive/MyDrive/ocrimages/thyrocare_0_657.jpg'
    ]
    main(image_names)




Output saved to: /content/drive/MyDrive/output.csv
